# Introducción a MPI

Bibliografía
==

<a href="http://computing.llnl.gov/tutorials/mpi/">Message Passing Interface (MPI)</a>

<a href="http://mpitutorial.com/tutorials/running-an-mpi-cluster-within-a-lan/">Running an MPI Cluster within a LAN</a>

<a href="http://condor.cc.ku.edu/~grobe/docs/intro-MPI-C.shtml">An introduction to the 
Message Passing Interface (MPI) using C</a>

<a href="http://condor.cc.ku.edu/~grobe/docs/intro-MPI.shtml">An introduction to the 
Message Passing Interface (MPI) using Fortran</a>



## Hello world


```c
//hello.c
#include <stdio.h>
#include <mpi.h>

int main(int argc, char **argv) 
{
  int ierr;

  ierr = MPI_Init(&argc, &argv);
  printf("Hello world\n"); 

  ierr = MPI_Finalize();
    return 0;
}
```

luego compilamos el archivo

```bash
mpicc hello.c -o hello
```

para finalmente ejecutarlo
```bash
mpirun -np 4 hello
```

la respuesta debería ser
```bash
Hello world
Hello world
Hello world
Hello world
```


## Identificar el número de procesadores y el procesador actual

Las funciones de mpi *MPI_Comm_rank* y *MPI_Comm_size* nos permiten identificar el id del procesador que estamos usando y la cantidad de procesadores que estamos usando, respectivamente. Acá un ejemplo:

```c
//hello2.c
#include <stdio.h>
#include <mpi.h>

int main(int argc, char **argv)
{
  int ierr, num_procs, my_id;

  ierr = MPI_Init(&argc, &argv);

  /* find out MY process ID, and how many processes were started. */

  ierr = MPI_Comm_rank(MPI_COMM_WORLD, &my_id);
  ierr = MPI_Comm_size(MPI_COMM_WORLD, &num_procs);

  printf("Hello world! I'm process %i out of %i processes\n", 
     my_id, num_procs);

  ierr = MPI_Finalize();
  
  return 0;
}
```

compilamos
```bash
mpicc hello2.c -o hello2
```

ejecutamos en este caso con 4 procesadores
```bash
mpirun -np 4 hello2
```

finalmente obtenemos como resultado lo siguiente

```bash

Hello world! I'm process 1 out of 4 processes
Hello world! I'm process 3 out of 4 processes
Hello world! I'm process 2 out of 4 processes
Hello world! I'm process 0 out of 4 processes

```

# Código en función del Rank

Este es un ejemplo de como identificar la variable rank y tomar desiciones en función de esta

```c
#include <mpi.h>
int main(int argc, char **argv)
   {
      int my_id, root_process, ierr, num_procs;
      MPI_Status status;

      ierr = MPI_Init(&argc, &argv); 
      ierr = MPI_Comm_rank(MPI_COMM_WORLD, &my_id);
      ierr = MPI_Comm_size(MPI_COMM_WORLD, &num_procs);

      if( my_id == 0 ) {

         /* do some work as process 0 */
      }
      else if( my_id == 1 ) {

         /* do some work as process 1 */
      }
      else if( my_id == 2 ) {

         /* do some work as process 2 */ 
      } 
      else {

         /* do this work in any remaining processes */
      }
      /* Stop this process */

      ierr = MPI_Finalize();
        
      return 0;
   }
```

compilación y ejecución
```bash
mpicc different_task.c -o different_task
mpirun -np 4 different_task
```

In [114]:
%%writefile hello.f
program hello_world
include "/usr/local/include/mpif.h"
include "/usr/local/include/mpif-config.h"
integer ierr

call MPI_INIT ( ierr )
print *, "Hello world"
call MPI_FINALIZE ( ierr )

stop
end

Overwriting hello.f


In [4]:
%%bash
#cat /usr/local/include/mpif.h

In [117]:
%%bash
gfortran -o hello hello.f -lmpi

/usr/local/include/mpif.h:54: Error: Can't open included file 'mpif-config.h'


# Démosle una oportunidad a python

"*La optimización prematura es la raíz de todos los males*"  Donald Knuth

Una ventaja de C es la velocidad de ejecución de las aplicaciones, una desventaja es que la ingeniería de software es demasiado costosa como para desarrollar aplicaciones complejas. Lo ideal es hacer un prototipo rápido en un lenguaje de ingeniería de software más ágil y luego de tener resultados favorables hacer una implementación que se oriente a la velocidad en los resultados.

En esta parte veremos algunos ejemplos de similares resultados a los anteriores pero en **python 2.7**

## Recursos

https://bitbucket.org/mpi4py/mpi4py

http://pythonhosted.org/mpi4py/mpi4py.pdf

http://pythonhosted.org/mpi4py/usrman/tutorial.html


In [56]:
%%writefile ej0.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    
    data = {'a': 7, 'b': 3.14}

    comm.send(data, dest=1, tag=11)

elif rank == 1:
    
    data = comm.recv(source=0, tag=11)
    print data


Overwriting ej0.py


In [57]:
%%bash
mpirun -np 2 python ej0.py

{'a': 7, 'b': 3.14}


In [120]:
%%writefile ej_non_blocking.py

#parecido al patron de diseno promesa
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    req.wait()
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    data = req.wait()
    print data

Overwriting ej_non_blocking.py


In [125]:
%%bash
mpirun -np 3 python ej_non_blocking.py

{'a': 7, 'b': 3.14}


In [1]:
%%writefile ej_numpy.py
from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(1000, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    data = numpy.empty(1000, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)
    print data

Overwriting ej_numpy.py


In [2]:
%%bash
mpirun -n 2 python ej_numpy.py

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.
  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.
  90.  91.  92.  93.  94.  95.  96.  97.  98.  99.]


## Clase 3: Aspectos de Collective comunication

### bibliografía
http://mpitutorial.com/tutorials/mpi-broadcast-and-collective-communication/
http://mpitutorial.com/tutorials/mpi-scatter-gather-and-allgather/

Lo que hay que entender del Collective comunication es sobre la comunicación entre nodos

### MPI_Bcast

![](http://mpitutorial.com/tutorials/mpi-scatter-gather-and-allgather/broadcastvsscatter.png)

Su uso es para enviar una copia de un objeto desde el nodo maestro a todos los esclavos, haremos un ejemplo de C y Python

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
        int rank;
        int buf;
        const int root=0;

        MPI_Init(&argc, &argv);
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

        if(rank == root) {
           buf = 777;
        }

        printf("[%d]: Before Bcast, buf is %d\n", rank, buf);

        /* common region between all nodes*/
        MPI_Bcast(&buf, 1, MPI_INT, root, MPI_COMM_WORLD);

        printf("[%d]: After Bcast, buf is %d\n", rank, buf);

        MPI_Finalize();
        return 0;
}
```

compilamos y ejecutamos

```bash
mpicc bcast1.c -o bcast1
mpirun bcast1
```

resultado

```bash
[0]: Before Bcast, buf is 777
[0]: After Bcast, buf is 777
[1]: Before Bcast, buf is 0
[2]: Before Bcast, buf is 0
[3]: Before Bcast, buf is 0
[1]: After Bcast, buf is 777
[2]: After Bcast, buf is 777
[3]: After Bcast, buf is 777
```


Podemos implementar en python el mismo código

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_rank()
rank = comm.Get_rank()

root = 0

buf = 0
if rank == root:
    buf = 777

print "[%d]: Before Bcast, buf is %d\n"%(rank, buf)
buf = comm.bcast(buf, root=root)
print "[%d]: After Bcast, buf is %d\n"%(rank, buf)
```

y ejecutamos
```bash
mpirun -n 4 python bcast1.py
```

cuyo resultado es
```bash
[2]: Before Bcast, buf is 0
[3]: Before Bcast, buf is 0
[0]: Before Bcast, buf is 777
[0]: Before Bcast, buf is 777
[1]: Before Bcast, buf is 0
[2]: Before Bcast, buf is 777
[1]: Before Bcast, buf is 777
[3]: Before Bcast, buf is 777
```

### MPI_Gather
Recoje los resultados de cada nodo y los envia al nodo maestro en forma de lista de resultados
![](http://mpitutorial.com/tutorials/mpi-scatter-gather-and-allgather/gather.png)

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_rank()
rank = comm.Get_rank()

root = 0

buf = 0
buf_list = None
if rank == root:
    buf = 777

print "[%d]: Before Bcast, buf is %d\n"%(rank, buf)
buf = comm.bcast(buf, root=root)
print "[%d]: Before Bcast, buf is %d\n"%(rank, buf)
buf = buf + rank
buf_list = comm.gather(buf, root=root)

if rank ==root:
    print buf_list
```

### Tarea 2

Existen dos formas de calcular la varianza de una muestra, por un lado la formula del momento central $$V[X]=E[(E[X]-X)^2]$$

A nivel muestral $$s^2=\frac{1}{n}\sum_{i=1}^n (\overline{x}-x_i)^2$$

Dada una muestra aleatoria uniforme de 100 datos calcule la media y la desviación estandar usando bcast y gather para un número arbitrario de nodos. No se debe usar send y recv

Los pasos son los siguientes.

Primeo se debe calcular el promedio en paralelo $\overline{x}$
se debe enviar ese valor $\overline{x}$ y la data $X$ una copia a cada nodo, para cada nodo se debe calcular $s^2=\sum_{i={i_{rank}}}^{n_{rank}} (\overline{x}-x_i)$ donde los limites inferior y superior se debe calcular dividiendo la data en función de la cantidad de nodos.

finalmente enviar todos los resultados al nodos root mediante gather y calcular la varianza de la muestra.


## La función MPI_Scatter
La función scatter es similar la función broadcast con la diferencia que no distribuye a cada nodo una copia del objeto si no que más bien solo una parte. Scatter toma una lista de objetos y distribuye a cada objeto el pedazo asociado al indice de la lista.

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
    print data
else:
    data = None
d = comm.scatter(data, root=0)

d += 1

data = comm.gather(d, root=0)

if rank == 0:
    print data
```



In [11]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    n = 1000
    rand = np.random.uniform(0,1,n)
    data = np.array_split(rand, size)
else:
    data = None
data_chunck = comm.scatter(data, root=0)

avg = np.mean(data_chunck)

avg = comm.gather(avg, root=0)

if rank == 0:
    avg = np.array(avg)
    print "gather es: ", avg
    print "el promedio de los promedios es: ", np.mean(avg)

[0.49705966039614502]


### Tarea 3
Desarrolle un algoritmo en MPI que calcule la media, y la varianza de una distribución $X\sim U(\theta_1=0,\)$ usando gather, scatter y broadcast.

### Tarea 4 Simulación de montecarlo para un juego de bingo

Considere el siguiente juego de facebook
www.facebook.com/WoBingo/
Desarrolle una simulación en paralelo que determine la distribución de la utilidad del jugador

# Ejercicios

1. Crear un programa en mpi-python que tome una lista de 9 elementos, particione esta lista en una partición de tres sublistas (partición). Envie cada una de estas listas a cada uno de los 3 nodos disponibles y encuentre una forma de calcular el promedio de la lista.

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

mean = None
lp = l1 = l2 = l3 = None

if rank == 0:
    l = [1.0, 4.0, 5.0, 20.0, 3.0, 2.0, 0.0, 1.0, 1.0]
    l1 = l[0:3]
    l2 = l[3:6]
    l3 = l[6:9]
    
    comm.send(l1, dest=1)
    comm.send(l2, dest=2)
    comm.send(l3, dest=3)

if rank == 1:
    lp = comm.recv(l1, source=0)
    
    mean = (lp[0] + lp[1] + lp[2]) / 3
    
    comm.send(mean, dest=0)
    
if rank == 2:
    lp = comm.recv(l2, source=0)
    
    mean = (lp[0] + lp[1] + lp[2]) / 3
    
    comm.send(mean, dest=0)
    
if rank == 3:
    lp = comm.recv(l3, source=0)
    
    mean = (lp[0] + lp[1] + lp[2]) / 3
    
    comm.send(mean, dest=0)

if rank == 0:
    
    mean0 = comm.recv(mean, source=1)
    mean1 = comm.recv(mean, source=2)
    mean2 = comm.recv(mean, source=3)
    
    mean_result = (mean0 + mean1 + mean2)/3
    print mean_result
```

# Depurando aplicaciones con mpi4py

## Referencias

https://bfroehle.com/2011/09/14/debugging-mpi-python/